# Basic functionality of octant package

Import the necessary modules

In [1]:
from pathlib import Path
from octant.core import TrackRun, OctantTrack, HOUR

Define the common data directory

In [2]:
sample_dir = Path('.') / 'sample_data'

Data are usually organised in hierarchical directory structure. Here, the relevant parameters are defined.

In [3]:
dataset = 'era5'
period = 'test'
run_id = 0

Construct the full path

In [4]:
track_res_dir = sample_dir / dataset / f'run{run_id:03d}' / period

## Load the data

In [5]:
TR = TrackRun(track_res_dir)

In [6]:
print(TR)

<octant.core.TrackRun>
[671 tracks]

Data columns:
lon | lat | vo | time | area | vortex_type | cat

Sources:
sample_data/era5/run000/test


The `TrackRun` object also has an HTML view available in Jupyter Notebooks

In [7]:
TR

<octant.core.TrackRun> [671 tracks]

In [8]:
type(TR)

octant.core.TrackRun

In [9]:
len(TR)

671

In [10]:
TR.tstep_h

1.0

### Concatenate data from several directories

In [11]:
# TR = TrackRun(one_directory)
# TR2 = TrackRun(another_directory)
# TR.extend(TR2)
## or
# TR += TrackRun(another_directory)

`TrackRun` class can be initialised empty:

In [12]:
tr_empty = TrackRun()

In [13]:
print(tr_empty)

<octant.core.TrackRun>
[0 tracks]


## Categorise tracks by different filters

In [14]:
import xarray as xr

In [15]:
lsm = xr.open_dataarray(sample_dir / dataset / 'lsm.nc')
lsm = lsm.squeeze()  # remove singular time dimension

In [16]:
TR.categorise(lsm=lsm, filt_by_dist=False, filt_by_land=True, filt_by_domain_bounds=True, coast_rad=50)

/local/denis/software/octant/octant/core.py:506: DeprecatedWarning: Use the new classify() function
  warnings.warn("Use the new classify() function", DeprecatedWarning)


In [17]:
TR

<octant.core.TrackRun> [671 tracks]

In [18]:
TR.size(), TR.size('basic'), TR.size('moderate')

(671, 135, 90)

## Units of TrackRun

In [19]:
import random

Each cyclone track stored in lists of `TrackRun` class as a `OctantTrack` instance

In [20]:
ot = random.choice([*TR['moderate'].groupby('track_idx')])[1]

It is essentially a sub-class of pandas.DataFrame

In [21]:
ot

lon   lat        vo                time         area  \
track_idx row_idx                                                          
346       0        24.3  71.4  0.000210 2011-01-18 09:00:00   1422.87744   
          1        24.9  71.4  0.000248 2011-01-18 10:00:00  11755.83203   
          2        23.4  72.0  0.000272 2011-01-18 11:00:00  18691.89258   
          3        24.9  71.7  0.000295 2011-01-18 12:00:00  21820.01953   
          4        25.5  71.7  0.000308 2011-01-18 13:00:00  22992.21289   
          5        27.3  71.4  0.000327 2011-01-18 14:00:00  24073.78125   
          6        27.6  71.4  0.000330 2011-01-18 15:00:00  26367.34961   
          7        28.2  71.4  0.000304 2011-01-18 16:00:00  19747.59375   

                   vortex_type  cat  
track_idx row_idx                    
346       0                  0    2  
          1                  0    2  
          2                  0    2  
          3                  0    2  
          4                  0    2  
          5                  0    2  
          6                  0    2  
          7                  0    2

In [22]:
type(ot)

octant.core.OctantTrack

It has a few useful properties

In [23]:
ot.lifetime_h

7.0

In [24]:
ot.total_dist_km

292.2882191179238

including maximum and mean vorticity in $ s^{-1}$:

In [25]:
ot.max_vort, ot.mean_vort

(0.00033004000000000003, 0.00028679125)

In [26]:
ot.gen_lys_dist_km

138.29617154948116

In [27]:
ot.average_speed

41.75545987398912

In [28]:
ot.lonlat

array([[24.3, 71.4],
       [24.9, 71.4],
       [23.4, 72. ],
       [24.9, 71.7],
       [25.5, 71.7],
       [27.3, 71.4],
       [27.6, 71.4],
       [28.2, 71.4]])

## octant's utilities

In [29]:
from octant.utils import great_circle

In [30]:
great_circle(lon1=9.6, lon2=10.2, lat1=76.9, lat2=78.9)

222826.50759451024